In [23]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [24]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00108456s, CPU 0.001049s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000467624s, CPU 0.000466s)
Info    : Meshing 3D.

In [25]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:HeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 386)


LowLevelFEM.Problem("body2", :HeatConduction, 3, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 3321)

In [26]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [27]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [28]:
FEM.initialTemperature!(problem, "left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [29]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [30]:
FEM.applyBoundaryConditions!(problem, K, C, q, [supp])

In [31]:
λmax = FEM.largestEigenValue(K, C)

29474.62441063024

In [32]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.248639216107802e-5 … 0.005655143580128274 0.00688188433604221; 0.0 0.00023887697581746192 … 0.012583050525775654 0.01402124972255328], [0.0, 0.001995734656097927, 0.003991469312195854, 0.005987203968293782, 0.007982938624391709, 0.009978673280489635, 0.011974407936587562, 0.013970142592685489, 0.015965877248783417, 0.017961611904881346  …  0.021953081217077203, 0.02394881587317513, 0.02594455052927306, 0.027940285185370988, 0.029936019841468917, 0.03193175449756684, 0.03392748915366477, 0.03592322380976269, 0.03791895846586062, 0.03991469312195854], Int64[], 21, :T)

In [33]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0008335803349351795 … 0.019271366519329338 0.020440644855610743; 0.0 0.004365347787506136 … 0.028044930838098267 0.029298012153639138], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T)

In [34]:
FEM.showDoFResults(problem, T, :T, visible=true)

0

In [35]:
q1 = FEM.solveHeatFlux(problem, T)

LowLevelFEM.VectorField([[0.0 -8.676311889067158e-15 … -1.3335568007227421e-6 -2.6873523601121593e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 -6.656013887802287e-30 … -5.955700410381799e-22 -1.1911400820763599e-21; 0.0 4.4373425918681914e-30 … 0.0 -5.955700410381799e-22], [0.0 -1.7791426939864756e-14 … -2.1186264475641013e-6 -4.155234312014839e-6; 0.0 -3.194886666145098e-28 … -4.7645603283054394e-21 1.4293680984916318e-20; … ; 0.0 0.0 … -5.955700410381799e-21 -4.7645603283054394e-21; 0.0 7.099748146989106e-29 … 0.0 -4.7645603283054394e-21], [0.0 -8.789233440404667e-14 … -5.54881477328442e-6 -1.0373077607970987e-5; 0.0 -3.8338639993741174e-27 … -9.529120656610879e-21 -1.334076891925523e-19; … ; 0.0 -2.129924444096732e-28 … -1.1911400820763599e-20 0.0; 0.0 -4.599088998576663e-28 … 1.5269698399627529e-21 -1.6292883476574546e-20], [0.0 -4.592722611852542e-13 … -1.5467988559098322e-5 -2.7640748303724767e-5; 0.0 -2.1583234366846883e-26 … 1.9058241313221758e-19 -3.8116482626443515e-20; … ; 0.0 3.299594224

In [36]:
FEM.showHeatFluxResults(problem, q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [37]:
q2 = FEM.solveHeatFlux(problem, T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -8.676311889067165e-15 … -1.3335568007227434e-6 -2.687352360112158e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 6.245004513516506e-16 … 1.8735013540549517e-15 -6.245004513516506e-16; 0.0 -1.5612511283791264e-16 … 6.245004513516506e-16 4.371503159461554e-15], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D)

In [38]:
FEM.showDoFResults(problem, q2, :qvec)

2

In [39]:
q3 = FEM.elementsToNodes(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -8.676311889067165e-15 … -1.3335568007227434e-6 -2.687352360112158e-6; 0.0 0.0 … 0.0 0.0; … ; 0.0 6.245004513516506e-16 … 1.8735013540549517e-15 -6.245004513516506e-16; 0.0 -1.5612511283791264e-16 … 6.245004513516506e-16 4.371503159461554e-15], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D)

In [40]:
FEM.showDoFResults(problem, q3, :qvec)

3

In [41]:
err1 = FEM.fieldError(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q3D)

In [42]:
FEM.showDoFResults(problem, err1, :vector)

4

In [43]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [44]:
gmsh.finalize()